In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import pydot
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
dataset = pd.read_csv("/ml-100k/u.data",sep='\t',names="user_id,item_id,rating,timestamp ,user_emb_id ,movie_emb_id".split(","))
#user_emb_id = user_id - 1
#movie_emb_id = movie_id - 1


FileNotFoundError: File b'/ml-100k/u.data' does not exist

In [ ]:
dataset.head()


In [ ]:
len(dataset.user_id.unique()), len(dataset.item_id.unique())

In [ ]:
dataset.user_id = dataset.user_id.astype('category').cat.codes.values
dataset.item_id = dataset.item_id.astype('category').cat.codes.values

In [ ]:
dataset.head()

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(dataset, test_size=0.2)

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
import keras
from IPython.display import SVG
from keras.optimizers import Adam
from keras.utils.vis_utils import model_to_dot
from keras.layers import Embedding, Reshape, merge, Dot, Dropout, Dense ,concatenate
n_users, n_movies = len(dataset.user_id.unique()), len(dataset.item_id.unique())
n_latent_factors = 3

In [ ]:
movie_input = keras.layers.Input(shape=[1],name='Item')
movie_embedding = keras.layers.Embedding(n_movies + 1, n_latent_factors, name='Movie-Embedding')(movie_input)
movie_vec = keras.layers.Flatten(name='FlattenMovies')(movie_embedding)

user_input = keras.layers.Input(shape=[1],name='User')
user_vec = keras.layers.Flatten(name='FlattenUsers')(keras.layers.Embedding(n_users + 1, n_latent_factors,name='User-Embedding')(user_input))
#change 1.3
prod = keras.layers.merge.dot([movie_vec ,user_vec] ,axes = 1, normalize=True)
#prod = keras.layers.merge.dot([movie_vec ,user_vec] ,axes = 1, normalize=False)#​prod = keras.layers.merge.dot([movie_vec ,user_vec] ,axes = 1, normalize=Falserod = keras.layers.merge.dot([movie_vec ,user_vec] ,axes = 1, normalize=False#​prod = keras.layers.merge.dot([movie_vec ,user_vec] ,axes = 1, normalize=False)
model = keras.Model([user_input, movie_input], prod)
model.compile('adam', 'mean_squared_error')

In [ ]:
prod

In [ ]:
model.summary()

In [ ]:
history = model.fit([train.user_id, train.item_id], train.rating, epochs=100, verbose=0)

In [ ]:
pd.Series(history.history['loss']).plot(logy=True)
plt.xlabel("Epoch")
plt.ylabel("Train Error")

In [ ]:
y_hat = np.round(model.predict([test.user_id, test.item_id]),0)
y_true = test.rating

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_true, y_hat)

In [ ]:
movie_embedding_learnt = model.get_layer(name='Movie-Embedding').get_weights()[0]
pd.DataFrame(movie_embedding_learnt).describe()

In [ ]:
user_embedding_learnt = model.get_layer(name='User-Embedding').get_weights()[0]
pd.DataFrame(user_embedding_learnt).describe()

In [ ]:
from keras.constraints import non_neg
movie_input = keras.layers.Input(shape=[1],name='Item')
movie_embedding = keras.layers.Embedding(n_movies + 1, n_latent_factors, name='NonNegMovie-Embedding', embeddings_constraint=non_neg())(movie_input)
movie_vec = keras.layers.Flatten(name='FlattenMovies')(movie_embedding)

user_input = keras.layers.Input(shape=[1],name='User')
user_vec = keras.layers.Flatten(name='FlattenUsers')(keras.layers.Embedding(n_users + 1, n_latent_factors,name='NonNegUser-Embedding',embeddings_constraint=non_neg())(user_input))

prod = keras.layers.merge.dot([movie_vec ,user_vec] ,axes = 1, normalize=False)
model = keras.Model([user_input, movie_input], prod)
model.compile('adam', 'mean_squared_error')

In [ ]:
history_nonneg = model.fit([train.user_id, train.item_id], train.rating, epochs=10, verbose=0)

In [ ]:
movie_embedding_learnt = model.get_layer(name='NonNegMovie-Embedding').get_weights()[0]
pd.DataFrame(movie_embedding_learnt).describe()

In [ ]:
n_latent_factors_user = 5
n_latent_factors_movie = 8

movie_input = keras.layers.Input(shape=[1],name='Item')
movie_embedding = keras.layers.Embedding(n_movies + 1, n_latent_factors_movie, name='Movie-Embedding')(movie_input)
movie_vec = keras.layers.Flatten(name='FlattenMovies')(movie_embedding)
movie_vec = keras.layers.Dropout(0.2)(movie_vec)


user_input = keras.layers.Input(shape=[1],name='User')
user_vec = keras.layers.Flatten(name='FlattenUsers')(keras.layers.Embedding(n_users + 1, n_latent_factors_user,name='User-Embedding')(user_input))
user_vec = keras.layers.Dropout(0.2)(user_vec)


concat = keras.layers.concatenate([movie_vec, user_vec], axis=-1)#(, mode='concat',name='Concat')
concat_dropout = keras.layers.Dropout(0.2)(concat)
dense = keras.layers.Dense(200,name='FullyConnected')(concat)
dropout_1 = keras.layers.Dropout(0.2,name='Dropout')(dense)
dense_2 = keras.layers.Dense(100,name='FullyConnected-1')(concat)
dropout_2 = keras.layers.Dropout(0.2,name='Dropout')(dense_2)
dense_3 = keras.layers.Dense(50,name='FullyConnected-2')(dense_2)
dropout_3 = keras.layers.Dropout(0.2,name='Dropout')(dense_3)
dense_4 = keras.layers.Dense(20,name='FullyConnected-3', activation='relu')(dense_3)


result = keras.layers.Dense(1, activation='relu',name='Activation')(dense_4)
adam = Adam(lr=0.005)
model = keras.Model([user_input, movie_input], result)
model.compile(optimizer=adam,loss= 'mean_absolute_error')

In [ ]:
model.summary()

In [ ]:
history = model.fit([train.user_id, train.item_id], train.rating, epochs=250, verbose=0)

In [ ]:
y_hat_2 = np.round(model.predict([test.user_id, test.item_id]),0)
print(mean_absolute_error(y_true, y_hat_2))

print(mean_absolute_error(y_true, model.predict([test.user_id, test.item_id])))